In [19]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from pathlib import Path
import numpy as np
import seaborn as sns

In [20]:
folder = "data/comp1_2020_1/"
Path(folder).mkdir(parents=True, exist_ok=True)

In [21]:
with open(f'{folder}/dropout_df_total_and_per_week_dict_3_6.pkl', 'rb') as pklfile:
    df_total_and_per_week = pickle.load(pklfile)

In [22]:
exclude = df_total_and_per_week[(df_total_and_per_week['attempts_12'] == 0) & \
                                (df_total_and_per_week['attempts_13'] == 0) & \
                                (df_total_and_per_week['attempts_14'] == 0) & \
                                (df_total_and_per_week['attempts_15'] == 0)]
dtw2 = df_total_and_per_week.drop(index=exclude.index)
dtw2

,attempts_12,success_rate_12,avg_timedelta_to_solution_12,avg_timedelta_from_beginning_12,on_time_exercises_12,frequency_12,success_count_12,attempts_13,success_rate_13,avg_timedelta_to_solution_13,...,on_time_exercises_19,frequency_19,success_count_19,attempts,success_rate,avg_timedelta_to_solution,avg_timedelta_from_beginning,on_time_exercises,frequency,success_count
910,8,0.375000,65.386857,328.720191,3,1,3,24,0.291667,291.476823,...,6,1,6,32,0.312500,223.649833,519.849833,10,2,10
911,4,0.750000,132.823499,684.156833,3,1,3,36,0.194444,23916.274449,...,6,2,6,40,0.250000,16781.239164,17381.439164,10,3,10
912,6,0.500000,24.270101,207.270101,3,1,3,7,1.000000,0.000000,...,6,1,6,13,0.769231,7.281030,141.581030,10,2,10
914,9,0.333333,430.621847,918.621847,3,1,3,91,0.076923,40044.526822,...,6,2,6,100,0.100000,28160.355329,28983.855329,10,4,10
915,8,0.375000,124.739469,243.072802,3,1,3,12,0.416667,18.759911,...,5,1,5,20,0.400000,58.502245,167.252245,8,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,0,NaN,NaN,NaN,0,3,0,0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,0,3,0
1403,0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,...,5,1,5,0,NaN,NaN,NaN,0,0,0
1405,6,0.500000,32.428669,655.095336,0,2,3,22,0.181818,26859.988079,...,0,0,0,28,0.250000,15362.462618,15919.319761,0,7,7
1407,56,0.053571,2124.487458,3170.154125,0,1,3,0,NaN,NaN,...,0,0,0,56,0.053571,NaN,NaN,0,3,3


In [23]:
dtw2['success_count_label'] = dtw2['success_count_17'] + dtw2['success_count_19']

In [24]:
# Features and fill missing data
features_week6 = dtw2[['on_time_exercises_12',
                       'on_time_exercises_13',
                       'on_time_exercises_14',
                      ]].reset_index(drop=True)

# Target
outcome_week6 = dtw2['success_count_label'].reset_index(drop=True)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(features_week6, outcome_week6, test_size=0.2, random_state=0)
idx = np.array(X_train.index.tolist())

In [26]:
X_train_const = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_const).fit()
X_test_const = sm.add_constant(X_test)
y_pred_ols = model.predict(X_test_const)
rmse = mean_squared_error(y_test, y_pred_ols, squared=False)
print("RMSE: %.2f" % rmse)
model.summary()

RMSE: 3.80


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     success_count_label   R-squared:                       0.301
Model:                             OLS   Adj. R-squared:                  0.295
Method:                  Least Squares   F-statistic:                     46.59
Date:                 Sun, 07 Aug 2022   Prob (F-statistic):           4.65e-25
Time:                         23:58:15   Log-Likelihood:                -926.29
No. Observations:                  328   AIC:                             1861.
Df Residuals:                      324   BIC:                             1876.
Df Model:                            3                                         
Covariance Type:             nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    4.5677      0.416     10.981      0.000       3.749       5.386
on_time_exercises_12     0.7315      0.186      3.933      0.000       0.366       1.097
on_time_exercises_13     0.2754      0.100      2.757      0.006       0.079       0.472
on_time_exercises_14     0.4524      0.088      5.114      0.000       0.278       0.626
==============================================================================
Omnibus:                       12.074   Durbin-Watson:                   1.874
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               10.810
Skew:                          -0.379   Prob(JB):                      0.00449
Kurtosis:                       2.536   Cond. No.                         12.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
with open("%s/model_week6.pkl" % folder, "wb") as pklfile:
    pickle.dump(model, pklfile)

In [28]:
n_medio = 0
medio = 0
n_baixo = 0
baixo = 0
n_alto = 0
alto = 0
for n in range(len(y_test)):
    if y_pred_ols.tolist()[n] < 6.5:
        baixo += 1
        if y_test.tolist()[n] < 4:
            n_baixo += 1
    elif y_pred_ols.tolist()[n] < 9.5:
        medio += 1
        if y_test.tolist()[n] < 4:
            n_medio += 1
    else:
        alto += 1
        if y_test.tolist()[n] < 4:
            n_alto += 1
    
if baixo:
    print(f'Risco baixo: {n_baixo}/{baixo} ({100*n_baixo/baixo}%)')
if medio:
    print(f'Risco médio: {n_medio}/{medio} ({100*n_medio/medio}%)')
if alto:
    print(f'Risco alto: {n_alto}/{alto} ({100*n_alto/alto}%)')

Risco baixo: 10/20 (50.0%)
Risco médio: 3/24 (12.5%)
Risco alto: 2/38 (5.2631578947368425%)


In [29]:
y_pred_ols.unique()

array([ 8.3160012 ,  4.56771724, 11.5808553 ,  6.82950393, 11.85621034,
        5.57078466, 10.04678099,  9.14206631,  9.77142595,  6.03072155,
        7.5572163 ,  8.31221142,  9.92872507,  6.49520251,  9.38634883,
       11.403853  , 10.57778788,  9.75172277,  8.41435394,  6.37714659,
       11.12849796,  9.12236313,  6.76222371,  9.94842825,  5.6691374 ,
        9.66170387,  6.95134963, 11.03014522, 11.12470818,  7.7342186 ,
        5.47243192, 10.84935314,  9.49607091,  9.22071587,  6.21984747,
        7.13214171, 10.85314292,  8.03685638,  8.48163416,  7.03757875,
        5.2992194 ,  8.76835854, 11.30550026])